In [1]:

import sys, os

import pygame
from evoman.environment import Environment
from demo_controller import player_controller

import numpy as np


pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# choose this for not using visuals and thus making experiments faster
headless = False

if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"


experiment_name = "Yossi GA"

if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

n_hidden_neurons = 10

# initializes simulation in individual evolution mode, for single static enemy.
env = Environment(
    experiment_name=experiment_name,
    enemies=  [1],
    playermode="ai",
    player_controller=player_controller(n_hidden_neurons),
    enemymode="static",
    level=2,
    speed="fastest",
    visuals=False,
)


MESSAGE: Pygame initialized for simulation.


In [3]:
def simulation(x):
    f,p,e,t = env.play(pcont=x)
    return f

def evaluate(x):
    return np.array(list(map(lambda y: simulation(y), x)))

def norm(x, pfit_pop):
    if (max(pfit_pop) - min(pfit_pop)) > 0:
        x_norm = (x - min(pfit_pop)) / (max(pfit_pop) - min(pfit_pop))
    else:
        x_norm = 0

    if x_norm <= 0:
        x_norm = 0.0000000001
    return x_norm

In [4]:
class GA:

    def __init__(self, n_genomes, population_size, n_offspring, mutation_p) -> None:
        self.n_genomes = n_genomes
        self.population_size = population_size
        self.n_offspring = n_offspring
        self.mutation_p = mutation_p
        
    ###################
    # INITIALIZING 
    ###################

    def initialize_population(self) -> np.array:
        """
            creates a population with random genomes

        Returns:
            np.array: _description_
        """

        return np.random.normal(size=(self.population_size, self.n_genomes))
    
    ###################
    # SELECTION
    ###################

    def tournament_selection(self, population: np.array, fitness : np.array) -> np.array:
        """
            Randomly samples two species from a population and returns the individual with the highest fitness score

        Args:
            population (np.array): An array that holds all individuals
            fitness (np.array): fitness values of all individuals

        Returns:
            np.array: the winning individual
        """

        index_p1, index_p2 = np.random.randint(0, self.population_size, 2)
        return population[index_p1] if fitness[index_p1] > fitness[index_p2] else population[index_p2]
    

    def eletist_selection(self, population: np.array, fitness : np.array, frac: float = .1) -> np.array:
        """
            Takes the fraction of the population with the best fitness score

        Args:
            population (np.array): An array that holds all individuals
            fitness (np.array): fitness values of all individuals
            frac (float, optional): The faction of the population. Defaults to .1.

        Returns:
            np.array: the top performing individuals
        """

        population_zip = zip(population, fitness)

        population_sorted = sorted(population_zip, key= lambda x : x[1], reverse=True)
        
        population_slice = int(np.floor(frac * self.population_size))
    
        return [pop for pop, _ in population_sorted][0:population_slice]

    ###################
    # MUTATION
    ###################

    def mutate(self, individual: np.array) -> np.array:
        """
            each genome of an individual is randomly mutated
        
        Args:
            individual (np.array): individual from population
            sigma (float, optional): sigma of normal distribution. Defaults to 1.0.

        Returns:
            np.array: mutated individual
        """

        mutation_dist = np.random.uniform(0, 1, size=self.n_genomes) <= .5
        individual += mutation_dist * np.random.normal(0, self.mutation_p, size=self.n_genomes)
        return individual
            

    ###################
    # CROSSOVER
    ###################

 
    def crossover(self, population: np.array, fitness : np.array) -> np.array:
        """
            Performs crossover based on tournament selection. Subsequently, applies mutation to random individuals

        Args:
            population (np.array): An array that holds all individuals
            fitness (np.array): fitness values of all individuals

        Returns:
            np.array: returns offspring
        """
        
        total_offspring = []

        for i in range(0, self.population_size, 2):

            indv_1 = self.tournament_selection(population, fitness)
            indv_2 = self.tournament_selection(population, fitness)

            offspring = np.zeros(shape = (self.n_offspring, self.n_genomes))
            
            for child in offspring:
                cross_distribution = np.random.uniform(0, 1) 
                child += cross_distribution * indv_1 + (1 - cross_distribution) * indv_2
                child = self.mutate(child)
                total_offspring.append(child)
                
        return np.asarray(total_offspring)
    


In [5]:
hyperp = {
    'n_vars': (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5,
    'population_size' : 150,
    'n_evaluations' : 4,
    'n_offspring' : 3,
    'mutation_sigma' : .5,
    'generations' : 5,
    'n_best': 1,
}


algo = GA( n_genomes=hyperp['n_vars'],
           population_size=hyperp['population_size'],
           n_offspring= hyperp['n_offspring'],
           mutation_p=hyperp['mutation_sigma'])


network = {
    'weights': [],
    'fitness': -99999999
}


# Initialize

population_w = algo.initialize_population()
population_f = evaluate(population_w)

# Evolve

print(5*'-' + f' Start Evolving ' + 5*'-', end='\n')


for generation in range(hyperp['generations']):
    
    # PARENT SELECTION + MUTATION | Select parrents and create a new generation
    offspring_w = algo.crossover(population_w, population_f)
    offspring_f = evaluate(offspring_w)

    # Combine the old generation with the new generation
    combined_w = np.vstack((population_w, offspring_w))
    combined_f = np.append(population_f, offspring_f)

    # min-max scale the fitness score such that you can use it as probabilities
    normalized_f = np.asarray(list(map(lambda x : norm(x, combined_f), combined_f)))

    # SURVIVOR SELECTION | select the best individuals from the population
    idx_n_best_individuals = np.argpartition(normalized_f, -hyperp['n_best'])[-hyperp['n_best']:]
    n_best_individuals_w = combined_w[idx_n_best_individuals]
    n_best_individuals_f = combined_f[idx_n_best_individuals]

    # remove those best from the population
    combined_w = np.delete(combined_w, idx_n_best_individuals, axis = 0)
    combined_f = np.delete(combined_f, idx_n_best_individuals, axis = 0)
    normalized_f = np.delete(normalized_f, idx_n_best_individuals, axis = 0)

    # Calculate a survival probability
    survival_prob = normalized_f / np.sum(normalized_f)

    # Select from population
    selection_idx = np.random.choice(combined_w.shape[0], (hyperp['population_size'] - hyperp['n_best']), p=survival_prob, replace=False)

    # Chose new population
    population_w = np.vstack((combined_w[selection_idx], n_best_individuals_w))
    population_f = np.append(combined_f[selection_idx], n_best_individuals_f)

    best_idx = np.argmax(population_f)
    best_w = population_w[best_idx]
    best_f = population_f[best_idx]

    if network['fitness'] < best_f: 
        network['weights'] = best_w
        network['fitness'] = best_f

    
    print('\t' + f'Generation: {generation} - Overall best: {network['fitness']} | Generations best: {best_f} | mean: {np.mean(population_f)}')

print(5*'-' + f' Finished Evolving ' + 5*'-', end='\n')

----- Start Evolving -----
	Generation: 0 - Overall best: 75.15356122494228 | Generations best: 75.15356122494228 | mean: 0.8186091006678472
	Generation: 1 - Overall best: 75.15356122494228 | Generations best: 75.15356122494228 | mean: 7.63637000533069
	Generation: 2 - Overall best: 75.15356122494228 | Generations best: 75.15356122494228 | mean: 20.83511603958326
	Generation: 3 - Overall best: 75.15356122494228 | Generations best: 75.15356122494228 | mean: 30.58129053811616
	Generation: 4 - Overall best: 75.15356122494228 | Generations best: 75.15356122494228 | mean: 35.58242079380314
----- Finished Evolving -----


# Run solution

In [11]:
# os.environ["SDL_VIDEODRIVER"] = ""

pygame.display.update()

env = Environment(
    experiment_name=experiment_name,
    enemies=  [2],
    playermode="ai",
    player_controller=player_controller(n_hidden_neurons),
    enemymode="static",
    level=2,
    speed="normal",
    visuals=True,
)


error: video system not initialized